## Setup

In [ ]:
try:
    TOKEN_NUMBER
except:
    TOKEN_NUMBER = 0
try:
    PROXY_NUMBER
except:
    PROXY_NUMBER = TOKEN_NUMBER

In [ ]:
API_PERIOD = 1
%run ApiSetup.ipynb

In [ ]:
with open(get_datapath(f"mal/mal_authentication/clientid.{TOKEN_NUMBER}.txt")) as f:
    MAL_ACCESS_TOKEN = f.readlines()[0].strip()

In [ ]:
def call_api(url):
    assert len(PROXYLIST) <= 1
    return call_api_internal(
        url,
        "GET",
        "mal",
        headers={"X-MAL-CLIENT-ID": MAL_ACCESS_TOKEN},
    )

In [ ]:
def get_user_media_list(username, mediatype):
    if mediatype == "anime":
        return get_user_anime_list(username)
    elif mediatype == "manga":
        return get_user_manga_list(username)
    else:
        raise ValueError()

## Anime lists

In [ ]:
def get_user_list(username, listtype, processfn):
    anime_lists = []
    more_pages = True
    url = f"https://api.myanimelist.net/v2/users/{username}/{listtype}list?limit=1000&fields=list_status&nsfw=true"
    while more_pages:
        response = call_api(url)
        if response.status_code in [403, 404]:
            # 403: This can occur if the user has privated their list
            # 404: This can occur if the user deleted their account
            logger.warning(f"Error {response} received when handling {url}")
            return pd.DataFrame(), False
        if not response.ok:
            logger.warning(f"Error {response} received when handling {url}")
            return pd.DataFrame(), False

        json = response.json()
        anime_lists.append(processfn(json))
        more_pages = "next" in json["paging"]
        if more_pages:
            url = json["paging"]["next"]
    user_anime_list = pd.concat(anime_lists, ignore_index=True)
    user_anime_list["username"] = username
    return user_anime_list, True

In [ ]:
def parse_anime_json_node(x):
    ls = x["list_status"]
    entry = pd.DataFrame.from_dict(
        {
            "uid": [x["node"]["id"]],
            "status": [ls.get("status", "")],
            "score": [ls.get("score", -1)],
            "num_episodes_watched": [ls.get("num_episodes_watched", -1)],
            "is_rewatching": [ls.get("is_rewatching", False)],
            "start_date": [ls.get("start_date", "")],
            "finish_date": [ls.get("finish_date", "")],
            "priority": [ls.get("priority", -1)],
            "num_times_rewatched": [ls.get("num_times_rewatched", -1)],
            "rewatch_value": [ls.get("rewatch_value", -1)],
            "updated_at": [ls.get("updated_at", "")],
        }
    )
    return entry


def process_anime_json(json):
    entries = [parse_anime_json_node(x) for x in json["data"]]
    if entries:
        return pd.concat(entries, ignore_index=True)
    else:
        return pd.DataFrame.from_dict(
            {
                "uid": [],
                "status": [],
                "score": [],
                "num_episodes_watched": [],
                "is_rewatching": [],
                "start_date": [],
                "finish_date": [],
                "priority": [],
                "num_times_rewatched": [],
                "rewatch_value": [],
                "updated_at": [],
            }
        )


def get_user_anime_list(username):
    return get_user_list(username, "anime", process_anime_json)

## Manga lists

In [ ]:
def parse_manga_json_node(x):
    ls = x["list_status"]
    entry = pd.DataFrame.from_dict(
        {
            "uid": [x["node"]["id"]],
            "status": [ls.get("status", "")],
            "score": [ls.get("score", "")],
            "num_volumes": [ls.get("num_volumes_read", "")],
            "num_chapters": [ls.get("num_chapters_read", "")],
            "updated_at": [ls.get("updated_at", "")],
        }
    )
    return entry


def process_manga_json(json):
    entries = [parse_manga_json_node(x) for x in json["data"]]
    if entries:
        return pd.concat(entries, ignore_index=True)
    else:
        return pd.DataFrame.from_dict(
            {
                "uid": [],
                "status": [],
                "score": [],
                "num_volumes": [],
                "num_chapters": [],
                "updated_at": [],
            }
        )


def get_user_manga_list(username):
    return get_user_list(username, "manga", process_manga_json)

## Anime

In [ ]:
def get_media_fields(media):
    if media == "anime":
        media_fields = [
            "num_episodes",
            "related_anime",
            "average_episode_duration",
            "source",
            "studios",
            "start_season",
        ]
    elif media == "manga":
        media_fields = ["num_volumes", "num_chapters", "related_manga", "authors"]
    else:
        assert False
    return [
        "id",
        "title",
        "main_picture",
        "alternative_titles",
        "start_date",
        "end_date",
        "synopsis",
        "num_list_users",
        "num_scoring_users",
        "nsfw",
        "genres",
        "media_type",
        "status",
        "recommendations",
    ] + media_fields

In [ ]:
def process_media_details_json(json, media):
    def get_key(key):
        return [json[key] if key in json else "NaN"]

    special_cols = {
        f"related_{media}": [
            [
                {f"{media}_id": x["node"]["id"], "relation": x["relation_type"]}
                for x in json[f"related_{media}"]
            ]
        ],
        "recommendations": [
            [
                {
                    f"{media}_id": x["node"]["id"],
                    "num_recommendations": x["num_recommendations"],
                }
                for x in json["recommendations"]
            ]
        ],
        "genres": [[x["name"] for x in json["genres"]] if "genres" in json else "[]"],
    }
    regular_cols = {
        x: get_key(x) for x in get_media_fields(media) if x not in special_cols
    }
    regular_cols.update(special_cols)
    df = pd.DataFrame.from_dict(regular_cols)
    df = df.rename({"id": f"{media}_id", "media_type": "medium"}, axis=1)
    return df


def get_media_details(mediaid, media):
    url = f'https://api.myanimelist.net/v2/{media}/{mediaid}?fields={",".join(get_media_fields(media))}&nsfw=true'
    response = call_api(url)
    if response.status_code == 404:
        logger.warning(f"Received error {response.status_code} while accessing {url}")
        return pd.DataFrame()
    try:
        response.raise_for_status()
    except Exception as e:
        logger.warning(f"Received error {str(e)} while accessing {url}")
        return pd.DataFrame()
    df = process_media_details_json(response.json(), media)
    return df